# 👻 Ghost on Local
---
[![Phantom](https://img.shields.io/badge/Version-2.0-teal.svg)]()
[![Phantom](https://img.shields.io/badge/Github-👻-black.svg)](https://github.com/Ming-doan/timeseries-imputation.git)

In [ ]:
import pandas as pd
from modules.alias import get_by_aliases
from modules.utils.preprocessing import Preprocessing
from modules.utils.missing import CreateMissingDataFrame
from modules import Trainer
from modules.models import *
from modules.utils.callbacks import SavePlot, Combined
from modules.utils.generator import WindowGenerator
from modules.utils.cache import Cache

In [ ]:
# Datasets
DATASET = 'PhuLien'
SELECTED_COLUMNS = ['WaterLevel']
# Missing values
MISSING_PERCENTAGE = 13
MISSING_GAPS = 1
SEED = 42
MODE = 'Random' # 'Random', 'Linear'
# Models
WINDOW_SIZE = 12
BATCH_SIZE = 1
MODELS = get_by_aliases(['rf'])

### Preprocessing

In [ ]:
import os
if not os.path.exists('results'):
    os.makedirs('results')

In [ ]:
# Read CSV
df = pd.read_csv(f'{DATASET}.csv')
df = df[SELECTED_COLUMNS]

In [ ]:
# Preprocessing data
preprocessing = Preprocessing()
df = preprocessing.flow(df)

In [ ]:
# Create missing data
creator = CreateMissingDataFrame(df, MISSING_PERCENTAGE, MISSING_GAPS, split_mode=MODE, seed=SEED, is_constant_missing=True)

In [ ]:
creator.plot(save_path='results/missing_data.png')

### Training

In [ ]:
# Training
trainer = Trainer(model=MODELS)
combined_callback = Combined(n_models=len(MODELS), combination_mode='data_per', df=creator, save_directory="results")
sp = SavePlot(n_models=len(MODELS), save_directory="results")
cache = Cache(live_cache=True)

for train_df, test_df in creator:
    train_gen = WindowGenerator(train_df, WINDOW_SIZE, BATCH_SIZE)
    test_gen = WindowGenerator(test_df, WINDOW_SIZE, BATCH_SIZE)

    trainer.train(train_gen, test_gen, callbacks=[sp, combined_callback], cache=cache)
    trainer.reset()

In [ ]:
combined_callback.metrics.metrics.to_csv('results/metrics.csv', index=False)
combined_callback.metrics.metrics

In [ ]:
for model in MODELS:
    print(f'{model.name}: {model.get_params()}')

In [ ]:
# Plot history
import matplotlib.pyplot as plt
for model in MODELS:
    if 'histories' in dir(model):
        try:
            fig, ax = plt.subplots(1, 2, figsize=(15, 5))
            ax[0].plot(model.histories[0].history['loss'], label='Pipeline')
            ax[0].legend()
            ax[1].plot(model.histories[1].history['loss'], label='Reverse Pipeline')
            ax[1].legend()
            fig.suptitle(f'Loss of {model.name}')
            plt.tight_layout()
        except Exception as e:
            print(e)
            print('\033[91mCannot plot history of model', model.name)